# Weather Features (Optional)


Optional: download and merge weather data into features for improved forecasting.

In [ ]:
# Environment setup
import sys, subprocess
from pathlib import Path

print('Python:', sys.executable)
repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
subprocess.run(['pip', 'install', '-e', str(repo_root)], check=True)

In [ ]:
import subprocess

subprocess.run(['python', '-m', 'gridpulse.data_pipeline.download_weather', '--out', 'data/raw', '--start', '2017-01-01', '--end', '2020-12-31'], check=True)
subprocess.run(['python', '-m', 'gridpulse.data_pipeline.build_features', '--in', 'data/raw', '--out', 'data/processed', '--weather', 'data/raw/weather_berlin_hourly.csv'], check=True)
